In [ ]:
import numpy as np
import pywt

import pickle as pic

In [ ]:
from utils import *
from templates import *
from data_loader import EntireTraceIterator

In [ ]:
traces_path = "..\\..\\acquisition\\1000000_maskshuffle_allrounds\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Masking-Shuffling.mat"
key_path = "..\\..\\acquisition\\1000000_maskshuffle_allrounds\\carto_eB4-Rnd-3-WhiteningAndFullFilter-Masking-Shuffling.log"

In [ ]:
NUM_TRACES = 1_000_000
TRACE_SIZE = 200_000
PER_DIVISION = 250_000
data_loader = EntireTraceIterator(traces_path, key_path, nr_populations=1, nr_scenarios=1, trace_size=TRACE_SIZE, traces_per_division=PER_DIVISION, parse_output="keyshares+perms")

# Compute 2nd order Haar wavelets

In [ ]:
for j, (_, traces, _, _) in enumerate(data_loader((0,), (0,))):
    traces = traces[0][0]
    for i in range(0, PER_DIVISION, PER_DIVISION // 10):
        wavelet = pywt.wavedec(traces[i:i + PER_DIVISION // 10], 'haar', level=2)[0]
        with open(f"wavelets_haar_2/wavelet_{j * PER_DIVISION + i}_{j * PER_DIVISION + i + PER_DIVISION // 10}.pic", "wb") as w:
            pic.dump(wavelet, w)
    del traces

In [ ]:
wavelets = np.zeros((739094, 50000), dtype=np.float16)
for div in range(0, wavelets.shape[0], PER_DIVISION // 10):
    filepath = f"wavelets_haar_2/wavelet_{div}_{div + PER_DIVISION // 10}.pic"
    with open(filepath, "rb") as r:
        wavelet = pic.load(r)
    wavelets[div:div+wavelet.shape[0]] = np.float16(wavelet)

In [ ]:
np.where(np.all(wavelets == 0, axis=1))

(array([499999], dtype=int64),)

In [ ]:
wavelets = np.concatenate((wavelets[:499999], wavelets[500000:]), axis=0)


In [ ]:
with open("../traces_and_splits/wavelets_haar_2.pic", "wb") as w:
    pic.dump(wavelets, w)